In [45]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *
from cell_utilities import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
run_distributed(command='sudo apt-get install -y \
  build-essential \
  cython \
  libmysqlclient-dev \
  libhdf5-dev \
  libxml2-dev \
  libxslt1-dev \
  openjdk-8-jdk \
  python-dev \
    python-h5py \
    python-mysqldb \
    python-vigra \
  python-wxgtk3.0 \
  python-zmq',
                    argument_type='single',
                   cluster_size=16)

wait_qsub_complete()


16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


In [33]:
run_distributed(command='cd /shared/CellProfiler/; sudo pip install --editable .',
                    argument_type='single',
                   cluster_size=16)

wait_qsub_complete()


16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


In [5]:
download_from_s3(CELLPROFILER_PIPELINE_FP)

In [38]:
script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells_cellprofiler_v2.py')

In [56]:
# for stack in all_nissl_stacks:
for stack in ['MD585']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    t = time.time()
    sys.stderr.write('Detect cells using Cell Profiler ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # 16 or 32 or 64 sections, 420 seconds
    # 128, 568
    # all 1300 seconds.

Detect cells using Cell Profiler ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 1226.845634 seconds


In [57]:
script_fp = os.path.join(REPO_DIR, 'cells', 'detect_cells.py')

rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 0
Child returned 1
0.36 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.36 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


In [ ]:
# for stack in all_nissl_stacks:
for stack in ['MD585']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

    _, _, _, _, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]
    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    t = time.time()
    sys.stderr.write('Detect cells ...')

    run_distributed(command='ROOT_DIR=/scratch/ %(script_path)s %(stack)s \'%%(filenames)s\''%\
                        {'script_path': script_fp, 
                        'stack': stack},
                        kwargs_list={'filenames': valid_filenames[first_idx_among_valid:last_idx_among_valid+1][:16]},
                        argument_type='list2',
                       cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 
    # 16, 45 sec.
    # 32, 85 sec.
    # 64, 141 sec.
    # 128, 201 sec.

Detect cells ...Child returned 0
16 nodes requested, 16 nodes available...Continuing


rm -f ~/stderr_*; rm -f ~/stdout_*


Jobs submitted. Use wait_qsub_complete() to check if they finish.
